# Supplementals: Weeks 9-10

In [1]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

import requests
import re
from lxml import html

from bs4 import BeautifulSoup

## APIs 101: What Exactly is an API? Part 1
https://www.youtube.com/watch?v=cpRcK4GS068

## Intro to Web Scraping with Python and Beautiful Soup
https://www.youtube.com/watch?v=XQgXKtPSzUI

In [2]:
# pip install bs4

In [3]:
import bs4

In [4]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [5]:
# myUrl = "https://www.newegg.com/p/pl?d=video+card"
myUrl = "https://www.newegg.com/p/pl?d=video+card&N=100006662"

Open up connection, grab the page:

In [6]:
# Opening up connection, grabbing the page
uClient = uReq(myUrl) # this step actually downloads the web page

Using `.read()` will dump everything, so assign it to something:

In [7]:
page_html = uClient.read() 

In [8]:
page_html[:500]

b'<!DOCTYPE html><html lang="en-us"><head><title>video card | Newegg.com</title><meta charSet="utf-8"/><meta http-equiv="content-type" content="text/html; charset=UTF-8"/><meta name="referrer" content="always"/><meta name="keywords" content="video card"/><meta name="description" content="Search Newegg.com for video card. Get fast shipping and top-rated customer service."/><meta property="og:image" content="https://c1.neweggimages.com/WebResource/Themes/2005/Nest/logo_424x210.png"/><meta property="'

Close the client:

In [9]:
uClient.close()

Parse the HTML by calling the soup function. Again, store to a variable:

In [10]:
# HTML parsing
page_soup = soup(page_html, "html.parser")

Look at the header:

In [11]:
page_soup.h1

<h1 class="page-title-text">Video Cards &amp; Video Devices</h1>

In [12]:
page_soup.p

<p class="item-msg"><a class="shipped-by-newegg" href="https://kb.newegg.com/knowledge-base/shipped-by-newegg/">Shipped by Newegg</a></p>

Traverse the HTML. Convert every graphics card into a line item in a CSV file:

In [13]:
page_soup.body.span

<span class="hid-text">Search</span>

Right click one of the cards and 'Inspect Element'  
\<div class="item-container"> == \$0   

Write a script on how to parse one graphics card,  
Then loop through and find all the class containers to parse out every card

In [14]:
# Grabs each product
containers = page_soup.findAll("div", {"class": "item-container"})

How many did it find?

In [15]:
len(containers)

36

See the HTML for the first card:

In [16]:
containers[0]

<div class="item-container"><a class="item-img" href="https://www.newegg.com/msi-geforce-gtx-1660-super-gtx-1660-super-gaming-x/p/N82E16814137476?Description=video card&amp;cm_re=video_card-_-14-137-476-_-Product"><img alt="MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660 SUPER GAMING X 6GB 192-Bit GDDR6 PCI Express 3.0 x16 HDCP Ready Video Card" src="https://c1.neweggimages.com/ProductImageCompressAll300/14-137-476-V01.jpg" title="MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660 SUPER GAMING X 6GB 192-Bit GDDR6 PCI Express 3.0 x16 HDCP Ready Video Card"/></a><div class="item-info"><div class="item-branding"><a class="item-brand" href="https://www.newegg.com/MSI/BrandStore/ID-1312"><img alt="MSI" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1312.gif" title="MSI"/></a><a class="item-rating" href="https://www.newegg.com/msi-geforce-gtx-1660-super-gtx-1660-super-gaming-x/p/N82E16814137476?IsFeedbackTab=true#scrollFullInfo" title="Rating + 5"><i class="rating rating-5"></i><span clas

Can paste this into Sublime Text editor and set the syntax to HTML to make it more legible.  Looks like there's an HTML highlight option in Jupyter text files as well.  
Copy paste into https://beautifier.io/ to prettify it, then paste back into text file.

Prototype the loop before building it:

In [17]:
container = containers[0]

In [18]:
container.div

<div class="item-info"><div class="item-branding"><a class="item-brand" href="https://www.newegg.com/MSI/BrandStore/ID-1312"><img alt="MSI" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1312.gif" title="MSI"/></a><a class="item-rating" href="https://www.newegg.com/msi-geforce-gtx-1660-super-gtx-1660-super-gaming-x/p/N82E16814137476?IsFeedbackTab=true#scrollFullInfo" title="Rating + 5"><i class="rating rating-5"></i><span class="item-rating-num">(97)</span></a></div><a class="item-title" href="https://www.newegg.com/msi-geforce-gtx-1660-super-gtx-1660-super-gaming-x/p/N82E16814137476?Description=video card&amp;cm_re=video_card-_-14-137-476-_-Product" title="View Details">MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660 SUPER GAMING X 6GB 192-Bit GDDR6 PCI Express 3.0 x16 HDCP Ready Video Card</a><ul class="item-features"><li><strong>Max Resolution:</strong> 7680 x 4320</li><li><strong>DisplayPort:</strong> 3 x DisplayPort 1.4</li><li><strong>HDMI:</strong> 1 x HDMI 2.0b</li><li><

In [19]:
container.div.div

<div class="item-branding"><a class="item-brand" href="https://www.newegg.com/MSI/BrandStore/ID-1312"><img alt="MSI" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1312.gif" title="MSI"/></a><a class="item-rating" href="https://www.newegg.com/msi-geforce-gtx-1660-super-gtx-1660-super-gaming-x/p/N82E16814137476?IsFeedbackTab=true#scrollFullInfo" title="Rating + 5"><i class="rating rating-5"></i><span class="item-rating-num">(97)</span></a></div>

In [20]:
container.div.div.a

<a class="item-brand" href="https://www.newegg.com/MSI/BrandStore/ID-1312"><img alt="MSI" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1312.gif" title="MSI"/></a>

In [21]:
container.div.div.a.img

<img alt="MSI" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1312.gif" title="MSI"/>

In [22]:
container.div.div.a.img["title"]

'MSI'

Add that to the script.

In [23]:
# for container in containers:
#     brand = container.div.div.a.img["title"]

In [24]:
# container.div.div.a

Wrong one. Let's use `.findall()` to locate the right tag:

In [25]:
title_container = container.findAll("a", {"class": "item-title"})

In [26]:
title_container

[<a class="item-title" href="https://www.newegg.com/msi-geforce-gtx-1660-super-gtx-1660-super-gaming-x/p/N82E16814137476?Description=video card&amp;cm_re=video_card-_-14-137-476-_-Product" title="View Details">MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660 SUPER GAMING X 6GB 192-Bit GDDR6 PCI Express 3.0 x16 HDCP Ready Video Card</a>]

In [27]:
title_container[0]

<a class="item-title" href="https://www.newegg.com/msi-geforce-gtx-1660-super-gtx-1660-super-gaming-x/p/N82E16814137476?Description=video card&amp;cm_re=video_card-_-14-137-476-_-Product" title="View Details">MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660 SUPER GAMING X 6GB 192-Bit GDDR6 PCI Express 3.0 x16 HDCP Ready Video Card</a>

In [28]:
title_container[0].text

'MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660 SUPER GAMING X 6GB 192-Bit GDDR6 PCI Express 3.0 x16 HDCP Ready Video Card'

Back to the loop:

In [29]:
# for container in containers:
#     brand = container.div.div.a.img["title"]
    
#     title_container = container.findAll("a", {"class": "item-title"})
#     product_name = title_container[0].text

Let's grab shipping:  
Inspect shipping element - \<li> tag, class="price-ship"

In [30]:
shipping_container = container.findAll("li", {"class": "price-ship"})

In [31]:
shipping_container

[<li class="price-ship">$45.00 Shipping</li>]

In [32]:
shipping_container[0]

<li class="price-ship">$45.00 Shipping</li>

In [33]:
shipping_container[0].text

'$45.00 Shipping'

Free shipping has extra spaces, newlines, etc., so lets strip that:

In [34]:
shipping_container[0].text.strip()

'$45.00 Shipping'

In [35]:
# for container in containers:
#     brand = container.div.div.a.img["title"]
    
#     title_container = container.findAll("a", {"class": "item-title"})
#     product_name = title_container[0].text
    
#     shipping_container = container.findAll("li", {"class": "price-ship"})
#     shipping = shipping_container[0].text.strip()

Let's print it out first to make sure it works:

In [36]:
# for container in containers:
#     # Errors out because some don't have the little image with the brand name
#     brand = container.div.div.a.img["title"]
    
#     title_container = container.findAll("a", {"class": "item-title"})
#     product_name = title_container[0].text
    
#     shipping_container = container.findAll("li", {"class": "price-ship"})
#     shipping = shipping_container[0].text.strip()
    
#     print("Brand:", brand)
#     print("Product:", product_name)
#     print("Shipping:", shipping)
#     print('-'*60)

Looks like one entry doesn't have that little brand image tag. Might have to pull the Brand from another spot :|

In [37]:
container.div.div.a.img["title"]

'MSI'

Save to a file:

In [38]:
# filename = 'products.csv'
# f = open(filename, "w")

# headers = "brand, product_name, shipping\n"

# f.write(headers)

# for container in containers:
#     brand = container.div.div.a.img["title"]
    
#     title_container = container.findAll("a", {"class": "item-title"})
#     product_name = title_container[0].text
    
#     shipping_container = container.findAll("li", {"class": "price-ship"})
#     shipping = shipping_container[0].text.strip()
    
#     print("brand:", brand)
#     print("product_name:", product_name)
#     print("shipping:", shipping)
# #     print('-'*60)
    
#     # Notice some of the names have commas in them; need to replace these
#     f.write(brand + "," + product_name.replace(",", "|") + "," + shipping + "\n")
 

In [39]:
# container2 = containers[18]
# container2

In [40]:
# container2.findAll("a", {"class": "item-img combo-img-0"})[0]

In [41]:
# container2.div.a.img['alt']

In [42]:
# container2.div.a['title']

In [43]:
container.div.div.a.img["title"]

'MSI'

In [44]:
container.div.div.a.img

<img alt="MSI" src="https://c1.neweggimages.com/Brandimage_70x28/Brand1312.gif" title="MSI"/>

In [45]:
# container2.div.a.text

In [46]:
# container2.div.a.text.partition(" ")[0]

Woo hoo!!

In [47]:
filename = 'products.csv'
f = open(filename, "w")

headers = "brand, product_name, shipping\n"

f.write(headers)

for container in containers:
    # Add in a try except because some don't have 'title' there
    try:
        brand = container.div.div.a.img["title"]
    except:
        brand = container.div.a.text.partition(" ")[0]
    
    title_container = container.findAll("a", {"class": "item-title"})
    product_name = title_container[0].text
    
    shipping_container = container.findAll("li", {"class": "price-ship"})
    shipping = shipping_container[0].text.strip()
    
    print("brand:", brand)
    print("product_name:", product_name)
    print("shipping:", shipping)
    print('-'*60)
    
    # Notice some of the names have commas in them; need to replace these
    f.write(brand + "," + product_name.replace(",", "|") + "," + shipping + "\n")
 

brand: MSI
product_name: MSI GeForce GTX 1660 SUPER DirectX 12 GTX 1660 SUPER GAMING X 6GB 192-Bit GDDR6 PCI Express 3.0 x16 HDCP Ready Video Card
shipping: $45.00 Shipping
------------------------------------------------------------
brand: XFX
product_name: XFX Radeon RX 580 DirectX 12 RX-580P8DFD6 XXX Edition 8GB 256-Bit GDDR5 PCI Express 3.0 CrossFireX Support Video Card
shipping: Free Shipping
------------------------------------------------------------
brand: GIGABYTE
product_name: GIGABYTE GeForce GTX 1660 SUPER DirectX 12 GV-N166SGAMING OC-6GD 6GB 192-Bit GDDR6 PCI Express 3.0 x16 ATX Video Card
shipping: Free Shipping
------------------------------------------------------------
brand: ASUS
product_name: ASUS GeForce GT 1030 2GB Phoenix Fan OC Edition HDMI DVI Graphics Card (PH-GT1030-O2G)
shipping: $8.99 Shipping
------------------------------------------------------------
brand: MSI
product_name: MSI GeForce GTX 1050 Ti DirectX 12 GTX 1050 Ti GAMING X 4G 4GB 128-Bit GDDR5 PCI 

Cool.

## Python Web Scraping with Beautiful Soup and Regex
https://www.youtube.com/watch?v=F1kZ39SvuGE

In [51]:
import requests
import re
from lxml import html

In [60]:
# # get the data
# data = html.parse('scrape.html')

In [61]:
# html.tostring(data).text

In [62]:
# data.text

In [79]:
# get the data
from bs4 import BeautifulSoup
f = open('scrape.html', encoding='utf8')
data = BeautifulSoup(f)
f.close()

Quick and dirty method:

In [63]:
# extract the phone numbers, emails, whatever
phones = re.findall(r'(\(?[0-9]{3}\)?(?:\-|\s|\.)?[0-9]{3}(?:\-|\.)[0-9]{4})', data.text)
emails = re.findall(r'([\d\w\.]+@[\d\w\.\-]+\.\w+)', data.text)

Try with random golf url (works with any site):

In [67]:
data = requests.get('https://www.mg-cc.org/club-information/club-contacts')

In [68]:
# extract the phone numbers, emails, whatever
phones = re.findall(r'(\(?[0-9]{3}\)?(?:\-|\s|\.)?[0-9]{3}(?:\-|\.)[0-9]{4})', data.text)
emails = re.findall(r'([\d\w\.]+@[\d\w\.\-]+\.\w+)', data.text)

In [69]:
print(phones, emails)

['(215) 886-3033', '(267) 241-3239', '(215) 361-2926', '(215) 886-3200'] ['mdwyer@mg-cc.org', 'morme@mg-cc.org', 'bfritz@mg-cc.org', 'bfritz@mg-cc.org', 'alazovitz@mg-cc.org', 'alazovitz@mg-cc.org', 'glenhart@mg-cc.org', 'glenhart@mg-cc.org', 'kbeck@mg-cc.org', 'kbeck@mg-cc.org', 'bdoheny@mg-cc.org', 'bdoheny@mg-cc.org', 'DiningManager@mg-cc.org', 'DiningManager@mg-cc.org', 'cgiampa@mg-cc.org', 'cgiampa@mg-cc.org', 'jccotto@mg-cc.org', 'sfalatek@mg-cc.org', 'selmarobinson@gmail.com', 'selmairobinson@gmail.com', 'freycc@npenn.org', 'freycc@npenn.org', 'kmurphy@mg-cc.org', 'kmurphy@mg-cc.org']


Now back to the example file.  
Scrape using bs4, by tag:

In [79]:
# get the data
from bs4 import BeautifulSoup
f = open('scrape.html', encoding='utf8')
data = BeautifulSoup(f)
f.close()

In [80]:
# soup = BeautifulSoup(data.text, 'html.parser')

In [77]:
# for tr in soup.find_all('tr'):
#     for td in tr.find_all('td'):
#         print(td.text)

In [82]:
for tr in data.find_all('tr'):
    for td in tr.find_all('td'):
        print(td.text)

Kennedy
Integer@erateget.co.uk
1-437-147-6843
Graham
magnis.dis@montesnascetur.ca
1-200-770-0178
Aristotle
amet@velquam.ca
1-807-310-0245
Cullen
Quisque.purus.sapien@Nuncsollicitudincommodo.org
1-518-961-3821
Samson
turpis.egestas@consequat.org
1-900-662-7207
Talon
faucibus.orci@erat.org
1-413-755-3413
Tad
malesuada.ut.sem@a.org
1-780-416-2930
Chadwick
velit.Pellentesque@nibh.net
1-307-148-6690
Orson
elit.Curabitur.sed@Nullaaliquet.ca
1-867-296-6130
Orson
nunc@lacusQuisque.com
1-395-373-3605
Austin
parturient@idantedictum.co.uk
1-237-577-7639
Dexter
ante.lectus@interdumSedauctor.com
1-924-289-1310
Hiram
Aenean@fermentumconvallisligula.org
1-372-683-8872
Austin
et.ultrices@lacusQuisque.com
1-640-499-3399
Kaseem
nulla.In.tincidunt@Vestibulumuteros.net
1-852-586-9376
Nero
justo@enimMaurisquis.net
1-891-939-5295
Salvador
imperdiet.non@nonluctus.co.uk
1-263-377-6148
Thor
tempor.diam.dictum@enim.org
1-531-539-3780
Holmes
enim.diam.vel@inconsectetuer.co.uk
1-423-339-4607
Bruno
Suspendisse.non

In [87]:
# get the data
from bs4 import BeautifulSoup
f = open('scrape.html', encoding='utf8')
soup = BeautifulSoup(f)
f.close()

In [88]:
# for tr in data.find_all('tr'):
#     for td in tr.find_all('td'):
#         print(td.text)
# Add each to a list:
for tr in soup.find_all('tr'):
    values =  [td.text for td in tr.find_all('td')]
    print(values)

[]
['Kennedy', 'Integer@erateget.co.uk', '1-437-147-6843']
['Graham', 'magnis.dis@montesnascetur.ca', '1-200-770-0178']
['Aristotle', 'amet@velquam.ca', '1-807-310-0245']
['Cullen', 'Quisque.purus.sapien@Nuncsollicitudincommodo.org', '1-518-961-3821']
['Samson', 'turpis.egestas@consequat.org', '1-900-662-7207']
['Talon', 'faucibus.orci@erat.org', '1-413-755-3413']
['Tad', 'malesuada.ut.sem@a.org', '1-780-416-2930']
['Chadwick', 'velit.Pellentesque@nibh.net', '1-307-148-6690']
['Orson', 'elit.Curabitur.sed@Nullaaliquet.ca', '1-867-296-6130']
['Orson', 'nunc@lacusQuisque.com', '1-395-373-3605']
['Austin', 'parturient@idantedictum.co.uk', '1-237-577-7639']
['Dexter', 'ante.lectus@interdumSedauctor.com', '1-924-289-1310']
['Hiram', 'Aenean@fermentumconvallisligula.org', '1-372-683-8872']
['Austin', 'et.ultrices@lacusQuisque.com', '1-640-499-3399']
['Kaseem', 'nulla.In.tincidunt@Vestibulumuteros.net', '1-852-586-9376']
['Nero', 'justo@enimMaurisquis.net', '1-891-939-5295']
['Salvador', 'imp

In [89]:
data = []
for tr in soup.find_all('tr'):
    values =  [td.text for td in tr.find_all('td')]
    data.append(values)

In [91]:
print(data)

[[], ['Kennedy', 'Integer@erateget.co.uk', '1-437-147-6843'], ['Graham', 'magnis.dis@montesnascetur.ca', '1-200-770-0178'], ['Aristotle', 'amet@velquam.ca', '1-807-310-0245'], ['Cullen', 'Quisque.purus.sapien@Nuncsollicitudincommodo.org', '1-518-961-3821'], ['Samson', 'turpis.egestas@consequat.org', '1-900-662-7207'], ['Talon', 'faucibus.orci@erat.org', '1-413-755-3413'], ['Tad', 'malesuada.ut.sem@a.org', '1-780-416-2930'], ['Chadwick', 'velit.Pellentesque@nibh.net', '1-307-148-6690'], ['Orson', 'elit.Curabitur.sed@Nullaaliquet.ca', '1-867-296-6130'], ['Orson', 'nunc@lacusQuisque.com', '1-395-373-3605'], ['Austin', 'parturient@idantedictum.co.uk', '1-237-577-7639'], ['Dexter', 'ante.lectus@interdumSedauctor.com', '1-924-289-1310'], ['Hiram', 'Aenean@fermentumconvallisligula.org', '1-372-683-8872'], ['Austin', 'et.ultrices@lacusQuisque.com', '1-640-499-3399'], ['Kaseem', 'nulla.In.tincidunt@Vestibulumuteros.net', '1-852-586-9376'], ['Nero', 'justo@enimMaurisquis.net', '1-891-939-5295'],

Look for items from tr elements with a particular class:

In [92]:
data = []
for tr in soup.find_all('tr', {'class':'special'}):
    values =  [td.text for td in tr.find_all('td')]
    data.append(values)

In [93]:
print(data)

[['Cullen', 'Quisque.purus.sapien@Nuncsollicitudincommodo.org', '1-518-961-3821'], ['Samson', 'turpis.egestas@consequat.org', '1-900-662-7207'], ['Talon', 'faucibus.orci@erat.org', '1-413-755-3413'], ['Dexter', 'ante.lectus@interdumSedauctor.com', '1-924-289-1310']]


Work with a table on the page (div class='special table'):

In [105]:
data = []
# just want to find one element, use .find
div = soup.find('div', {'class': 'special_table'})
for tr in div.find_all('tr'):
    values =  [td.text for td in tr.find_all('td')]
    data.append(values)
    
print(data)

[[], ['Kennedy', 'Integer@erateget.co.uk', '1-437-147-6843'], ['Graham', 'magnis.dis@montesnascetur.ca', '1-200-770-0178'], ['Aristotle', 'amet@velquam.ca', '1-807-310-0245']]


Try scraping leaderboards on UMG Gaming:  
Extract: Place, Username, XP

In [106]:
# get the data
url = "https://umggaming.com/leaderboards"
data = requests.get(url)

# load data into bs4
soup = BeautifulSoup(data.text, 'html.parser')

In [115]:
leaderboard = soup.find('table', {'id': 'leaderboard-table'})
# print(leaderboard)
tbody = leaderboard.find('tbody')
# print(tbody)

In [118]:
# for tr in tbody.find_all('tr'):
#     print(tr)
#     print('-'*60)

In [119]:
for tr in tbody.find_all('tr'):
    place = tr.find_all('td')[0].text.strip()
    print(place)

1st
2nd
3rd
4th
5th
6th
7th
8th
9th
10th
11th
12th
13th
14th
15th
16th
17th
18th
19th
20th
21st
22nd
23rd
24th
25th


Get username, which is in the second 'td' and is the text inside that:

In [120]:
for tr in tbody.find_all('tr'):
    place = tr.find_all('td')[0].text.strip()
    username = tr.find_all('td')[1].find_all('a')[1].text.strip()
    print(place, username)

1st Hidro
2nd Baker
3rd Gunservative
4th Mikey
5th Tanntrix
6th ReconsPOV
7th x2Pac ThuGLorD
8th Rules
9th Rallied
10th Olly
11th TKhan
12th MuTeX
13th TMTwerk
14th Innocent
15th ihavok
16th stridox
17th iLLeY
18th exoficial
19th TeddyRecKs
20th C
21st UsernameChange12911
22nd Monstrous
23rd xAmpz
24th Trichoblast
25th Saggio


In [121]:
for tr in tbody.find_all('tr'):
    place = tr.find_all('td')[0].text.strip()
    username = tr.find_all('td')[1].find_all('a')[1].text.strip()
    # XP is in 5th 'td'
    xp = tr.find_all('td')[4].text.strip()
    print(place, username, xp)

1st Hidro 81553
2nd Baker 78608
3rd Gunservative 75717
4th Mikey 51208
5th Tanntrix 49071
6th ReconsPOV 44956
7th x2Pac ThuGLorD 44041
8th Rules 41415
9th Rallied 40188
10th Olly 39312
11th TKhan 38772
12th MuTeX 35607
13th TMTwerk 35025
14th Innocent 34897
15th ihavok 34881
16th stridox 34851
17th iLLeY 34541
18th exoficial 34057
19th TeddyRecKs 33254
20th C 33096
21st UsernameChange12911 32795
22nd Monstrous 32637
23rd xAmpz 32428
24th Trichoblast 32165
25th Saggio 31984


To sum it up:

In [122]:
# get the data
url = "https://umggaming.com/leaderboards"
data = requests.get(url)

# load data into bs4
soup = BeautifulSoup(data.text, 'html.parser')

leaderboard = soup.find('table', {'id': 'leaderboard-table'})
tbody = leaderboard.find('tbody')

for tr in tbody.find_all('tr'):
    place = tr.find_all('td')[0].text.strip()
    username = tr.find_all('td')[1].find_all('a')[1].text.strip()
    xp = tr.find_all('td')[4].text.strip()
    print(place, username, xp)

1st Hidro 81553
2nd Baker 78608
3rd Gunservative 75717
4th Mikey 51208
5th Tanntrix 49071
6th ReconsPOV 44956
7th x2Pac ThuGLorD 44041
8th Rules 41415
9th Rallied 40188
10th Olly 39312
11th TKhan 38772
12th MuTeX 35607
13th TMTwerk 35025
14th Innocent 34897
15th ihavok 34881
16th stridox 34851
17th iLLeY 34541
18th exoficial 34057
19th TeddyRecKs 33254
20th C 33096
21st UsernameChange12911 32795
22nd Monstrous 32637
23rd xAmpz 32428
24th Trichoblast 32165
25th Saggio 31984
